<p>
<font size='5' face='Georgia, Arial'>IIC2115 - Programación como herramienta para la ingeniería</font><br>
</p>

# Structured Query Language (SQL)

SQL es el lenguaje de bases de datos relacionales más ampliamente utilizado, tanto en la academia con en la industria. Aunque no todos los DBMS soportan el último estándar SQL completo (SQL1999), la mayoría de los productos soportan versiones muy cercanas en cuanto a los elementos discutidos en este curso.

SQL tiene varios aspectos relevantes, pero acá nos centraremos en los dos principales:
* Lenguage de definición de datos (DDL)
* Lenguaje de manipulación de datos (DML) 



## Definición de datos (DDL)

Este subconjunto de SQL soporta la creación, eliminación y modificación de definiciones de tablas. Las restricciones de integridad se pueden definir en tablas, ya sea cuando se crea la tabla o posteriormente.


### Creación y modificación de relaciones

El lenguaje SQL utiliza la palabra clave *table* para denotar una relación. Por simplicidad, sólo consideramos dominios que están incorporados en tipos de datos nativos, como INTEGER o STRING. La sentencia **CREATE TABLE** se utiliza para definir una nueva tabla. Para crear la relación Estudiantes, podemos usar la siguiente declaración:

> CREATE TABLE Estudiantes (id CHAR(30), nombre VARCHAR(20), login CHAR(20), edad (INTEGER), promedio (REAL))

Las tuplas pueden insertarse utilizando el comando **INSERT**. Por ejemplo, para insertar una única tupla en la relación Estudiantes, podemos realizar la siguiente declaración:

> INSERT INTO Estudiantes (id, nombre, login, edad, promedio) VALUES (53688, 'Hernán', 'Hfvaldivieso', 20, 5.5)

Opcionalmente, podemos omitir los nombres de columnas en la declaración, pero se considera buena práctica el incluirlos de manera explícita.

Adicionalmente, podemos borrar tuplas utilizando el comando **DELETE**:

> DELETE FROM Estudiantes E WHERE E.nombre = 'Hernán'

Para modificar el valor de las columnas de una tupla existente, usamos el comando **UPDATE**:

> UPDATE Estudiantes E SET E.edad = E.edad + 1, E.promedio = E.promedio - 1 WHERE E.id = 53688

Este ejemplo ilustra algunos puntos importantes. Se aplica la cláusula WHERE primero y determina qué filas se van a modificar. La cláusula SET determina cómo se van a modificar estas filas. Si la columna que se está modificando es también utilizada para determinar el nuevo valor, el valor utilizado en la expresión en el lado derecho del igual (=) es el valor antiguo, es decir, antes de la modificación. 

Finalmente, si se busca modificar la estructura de una tabla, es posible hacerlo mediante el comando **ALTER**:

> ALTER TABLE Estudiantes ADD COLUMN carrera VARCHAR(30)

En el caso anterior, se agregó una nueva columna a la tabla Estudiantes, mientras que si se desea eliminar una columna ya existente, se debe intercambiar **ADD** por **DROP**, y eliminar la descripción del tipo de dato (VARCHAR(30) en este caso). Cabe destacar que en ambos casos, la aparición de la palabra clave **COLUMN** es opcional.


### Restricciones de llave primaria

En SQL, podemos declarar que un subconjunto de las columnas de una tabla constituyen una llave, utilizando la restricción **UNIQUE**. A lo sumo, una de estas llaves candidatas puede ser declarada como primaria, utilizando la restricción **PRIMARY KEY**. Revisemos nuestra definición de la table Estudiantes, especificando ahora la restricción de llave:

> CREATE TABLE Estudiantes(id CHAR(20), nombre VARCHAR(30), login CHAR(20), edad INTEGER, promedio REAL,<br> 
> &emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&nbsp;UNIQUE(nombre, edad), CONSTRAINT LlaveEstudiante PRIMARY KEY(id))

Esta definición indica que **id** es la llave primaria y la combinación de nombre y edad es también una llave. La definición de la llave primaria también ilustra cómo podemos nombrar una restricción precediéndola con **CONSTRAINT**, lo que permite que, si se viola la restricción, se devuelve el nombre de la restricción y se puede utilizar para identificar el error más fácilmente.


### Restricciones de llave foránea

Definamos nuevamente la relación **Inscrito(id_estudiante: STRING, id_curso: STRING, nota: REAL)**:

> CREATE TABLE Inscrito(id_estudiante CHAR(20), id_curso CHAR(20),nota REAL,<br> 
> &emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&nbsp;PRIMARY KEY (id_estudiante, id_curso), FOREIGN KEY (id_estudiante) REFERENCES Estudiantes)

La restricción de llave foránea indica que cada valor de **id_estudiante** en **Inscrito**, también debe
aparecen en **Estudiantes**, es decir es una clave extranjera que hace referencia a **Estudiantes**. Específicamente, cada valor de **id_estudiante** en **Inscrito** debe aparecer como el valor en
el campo clave principal, **id**, de **Estudiantes**. Cabe notar además que la restricción de llave primaria
para **Inscrito** indica que un estudiante tiene exactamente una nota para cada curso. Si queremos grabar más de una nota por alumno, deberíamos cambiar la restricción de llave primaria.

## Manipulación de datos (DML)

Este subconjunto de SQL permite declarar consultas que se ejecutarán sobre una o más tablas.

### Estructura básica de una consulta

La estructura básica de una consulta en SQL es la siguiente:

> SELECT [DISTINCT] select-list <br>
> FROM from-list <br>
> WHERE qualification <br>

Cada consulta debe tener una cláusula **SELECT**, que especifica las columnas que deben conservarse
en el resultado, y una cláusula **FROM**, que especifica un producto cruzado de tablas. La
cláusula opcional **WHERE** especifica las condiciones de selección en las tablas mencionadas en
la cláusula FROM. Consideremos a continuación un ejemplo sencillo, encontrar los nombres y edades de todos los estudiantes:

> SELECT DISTINCT E.nombre, E.edad <br>
> FROM Estudiantes E

La respuesta a la consulta anterior es un conjunto de filas, cada una de las cuales es un par (nombre, edad). Si dos o más estudiantes tienen el mismo par (nombre, edad) la respuesta contendrá sólo un par de estos. Si omitimos el modificador **DISTINCT**, obtendríamos una copia de la fila, para cada estudiante con el mismo nombre y edad. Esto se conoce como un *multiconjunto*, que a diferencia de un conjunto tradicional, puede contener valores repetidos. 

La siguiente consulta agrega una restricción simple a la consulta anterior:

> SELECT *
> FROM Estudiantes E <br>
> WHERE E.promedio > 5.0

Existen dos diferencia principales entre esta consulta y la anterior. Primero, dado que queremos obtener todos los datos de un alumno, no es necesario especificar cada columna, ya que basta con ingresar un \*. Segundo, dentro de los registros de la tabla Estudiantes, sólo retornaremos aquellos que tienen un promedio superior a 5.0.

Consideremos ahora la sintaxis de una consulta SQL básica con más detalle:

* La from-list en la cláusula FROM es una lista de nombres de tabla. Un nombre de tabla puede ser seguido por una variable de rango; una variable de rango es particularmente útil cuando el mismo nombre de la tabla aparece más de una vez en la from-list.
* La select-list es una lista de nombres de columnas de tablas nombradas en la from-list.
* Los argumentos en la cláusula WHERE son una combinación booleana (es decir, una expresión utilizando los conectivos lógicas AND, OR y NOT) de condiciones con forma *expresión op expresión*, donde *op* es uno de los siguientes operadores (<, <=, =, <>,> =,>). Una expresión es un nombre de columna, un constante o una expresión aritmética o de string.


### Consultas básicas

A continuación veremos consultas de ejemplo. Nuestro primer ejemplo ilustra que el uso de variables de rango es opcional, a menos que sean necesarias para resolver una ambigüedad:

> SELECT E.nombre  <br>
> FROM Estudiantes E, Estudiantes_Postgrado P  <br>
> WHERE E.id = P.id AND edad >= 24

Como se aprecia, sólo se deben calificar las ocurrencias de **id**, ya que esta columna aparece en ambas tablas. Por el contrario, **edad**, al estar sólo en la tabla **Estudiantes**, no necesita calificación.

Una situación más compleja se puede ver en la siguiente consulta, donde buscamos los cursos en que está inscrito cada estudiante:

> SELECT E.nombre, C.nombre  <br>
> FROM Estudiantes E, Cursos C, Inscrito I  <br>
> WHERE E.id = I.id_estudiante AND C.id = I.id_curso

Esta consulta contiene una combinación de tres tablas, ya que es necesario pasar por la tabla **Inscrito** para relacionar a los estudiates con sus cursos.

La última consulta básica que revisaremos involucra el ordenamiento de los resultados de manera transparente para el usuario:

> SELECT E.nombre  <br>
> FROM Estudiantes E, Estudiantes_Postgrado P  <br>
> WHERE E.id = P.id AND edad >= 24  <br>
> ORDER BY edad ASC

Como se puede ver, la consulta es una extensión de la primera consulta básica revisada, donde ahora los resultados se ordenarán de menor a mayor, en base a la edad de los estudiantes.


### Agregación

Además de simplemente recuperar datos, a menudo queremos realizar algún cálculo o agregación de estos. A continuación, revisaremos una poderosa clase de constructos para calcular valores agregados como MIN y SUM. SQL soporta cinco operaciones agregadas, que pueden aplicarse en cualquier columna, digamos A, de una relación:

* COUNT ([DISTINCT] A): La cantidad de valores (unicos) en la columna A.
* SUM ([DISTINCT] A): La suma de todos los valores (unicos) en la columna A.
* AVG ([DISTINCT] A): El promedio de todos los valores (unicos) en la columna A.
* MAX (A): El máximo valor en la columna A.
* MIN (A): El mínimo valor en la columna A.

Veamos a continuación un ejemplo simple:

> SELECT AVG (E.edad)  <br>
> FROM Estudiantes E

La consulta anterior permite calcular de manera eficiente el promedio de las edades de todos los alumnos registrados. Si deseamos filtrar un poco más los resultados, podemos realizar la siguiente consulta :

> SELECT AVG (E.edad)  <br>
> FROM Estudiantes E  <br>
> WHERE E.promedio > 5.0

Cabe destacar que todos los elementos vistos anteriormente para la cláusula WHERE, pueden ser usados en conjunto con las funciones de agregación.


### Agrupación

Hasta ahora, hemos aplicado operaciones agregadas a todas las filas de una relación. A menudo, queremos aplicar operaciones agregadas a ciertos grupos de filas en una relación, donde el número de grupos depende de la instancia de relación (es decir, no se conoce de antemano). Por ejemplo, si queremos calcular el promedio de edad de los estudiantes, agrupados por sus promedios, deberíamos realizar algo por el estilo:

> SELECT AVG (E.edad)  <br>
> FROM Estudiantes E  <br>
> WHERE E.promedio = i

con i = 1,.., 7 (asumiendo notas enteras, como se indica en el esquema de la relación). Claramente, escribir 7 veces la misma consulta es tedioso, pero más importante aún, no siempre se conoce a priori el número de categorías por los que se quiere agrupar.

Para facilitar este tipo de problemas, SQL incluye una extensión a las consultas básicas en la forma de la cláusula **GROUP BY** (y el filtro opcional **HAVING**). Usando esta nueva funcionalidad, podemos escribir la consulta anterior de la siguiente manera:

> SELECT AVG(E.edad) <br>
> FROM Estudiantes E  <br>
> GROUP BY E.promedio <br>
> HAVING E.promedio >= 5


### Matching de patrones en strings

El último tipo de consulta que revisaremos son bastante comunes, ya que involucran la búsqueda sobre strings, con patrones que no necesariamente calzan completamente con la información almacenada. Por ejemplo, si actualmente quisieramos buscar todos los alumnos cuyos nombre comiencen con "MA", sería necesario realizar una cantidad enorme de consultas, considerando todas las posibles combinaciones. Afortunamente, SQL entrega un mecanismo simple para solucionar esta situación:

> SELECT \*  <br>
> FROM Estudiantes E  <br>
> WHERE E.nombre LIKE 'MA%'

Existen multiples patrones de strings que pueden verificarse, no solo patrones de inicio. Más información sobre esto puede encontrarse [acá](https://www.w3schools.com/sql/sql_like.asp)